In [5]:
# huggingface network problem, you'd better use VPN
# On V100, you can just use xch's port

import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7991'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7991'

In [6]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("/home/xuechenhao/hugginface/opt-6.7b")

# Use GPT2 tokenizer to tokenize the text
tokenizer = AutoTokenizer.from_pretrained("gpt2")


In [7]:
from datasets import load_dataset

nsamples = 128

traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')

trainenc = tokenizer("\n\n".join(traindata['text']), return_tensors='pt')
testenc = tokenizer("\n\n".join(testdata['text']), return_tensors='pt')

for _ in traindata:
    print(_)
    break

Token indices sequence length is longer than the specified maximum sequence length for this model (2428601 > 1024). Running this sequence through the model will result in indexing errors


{'text': ''}


In [11]:
trainenc.input_ids.shape

torch.Size([1, 2428601])

In [12]:
nsample = 1024
seqlen = 1024

trainloader = []

for i in range(nsamples):
    inp = trainenc.input_ids[:, i * seqlen: (i+1) * seqlen]
    tar = inp.clone()
    tar[:, :-1] = -100  # A typical masking operation, I don't know exactly why
    trainloader.append((inp, tar))

# Trainloader is all you need

In [16]:
# Try printing the first sentense
print(tokenizer.decode(trainloader[0][0][0]))



 = Valkyria Chronicles III = 




 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3 ), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. Employing the same fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". 


 The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers.